#IMPORT MODELLI

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

import warnings
warnings.filterwarnings('ignore')

#IMPORT FILE

In [ ]:
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/data/esercizi/sonar.csv  #con!wget + percorso non devo ogni volta caricare il file

In [ ]:
df = pd.read_csv('DAX Dati Storici.csv',index_col = 0) assegno già l'indice
df = pd.read_csv('DAX Dati Storici.csv')

In [ ]:
df = pd.read_csv('sonar.csv', header=None) #Se il file non ha l'intestazione delle colonne
df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class']     # assegno le colonne

In [ ]:
#creazione dataframe manualmente
data = {'product_name': ['laptop', 'printer', 'tablet', 'desk', 'chair'],           
        'price': [1200, 150, 300, 450, 200]}
df = pd.DataFrame(data, index=['product_1','product_2','product_3','product_4','product_5'])

# DATAFRAME CLEANING

In [ ]:
df.info()
df.describe().T
df.head()    # prime 5 righe
df.tail()    # ultime 5 righe

In [ ]:
df=df.sort_index(ascending=False)  #cambio ordinamento del dataframe in base all'indice
df.index = pd.to_datetime(df['Data'], dayfirst=True)  # quando ho dei timeseries  imposto index alla data
df = df.set_index('Payment ID')  # assegno l'indice a una colonna
df.rename(columns={"Data":"data"}, inplace=True)   # modifico il nome di una colonna
df = df.dropna()  # elimino tutte le righe che contengono valori nulli
df.dropna(subset=['Data', 'Codice posizione'], inplace=True)   # voglio applicare dropna() solo a determinate colonne
df = df.drop("data",axis=1)  # elimino una colonna intera
df.drop(df.loc[df['Vol']==0].index, inplace=True)    # elimino le righe di una colonna che hanno un determinato valore (0)
df["Gap"]=df["D+1 Apertura"]-df["Apertura"]    # creo una colonna da altre due facendo un'operazione
df.isnull().sum() # controllo i valori nulli
df.isnull().sum().sum()    # mi dice il totale dei valori nulli
df['Embarked'].isnull().sum()  # ricerca valori nulli in determinate colonne

import missingno as msno   # creo il grafico nero con la distribuzione dei valori nulli
msno.matrix(df)

In [ ]:
#se ci sono dati numerici continui si può usare la mediana o la media
#se ci sono varialbii categoriche meglio usare la moda

In [ ]:
df['Gender'].value_counts()    # vedo le occorrenze della colonna

In [ ]:
# Mean of Age column
df['Age'].mean()
df['Age'].median()
df['Age'].mode()

In [ ]:
# Replace Null Values (np.nan) with mean
df['Age'] = df['Age'].replace(np.nan, df['Age'].mean())
# Alternate way to fill null values with mean
df['Age'] = df['Age'].fillna(df['Age'].mean())

In [ ]:
# In the same way we can impute using median and mode
df['Age'] = df['Age'].replace(np.nan, df['Age'].median())
df['Age'] = df['Age'].replace(np.nan, df['Age'].mode())
# Alternate ways to impute null values with median and mode
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Age'] = df['Age'].fillna(df['Age'].mode())

In [ ]:
# Backward fill or Forward fill can be used to impute the previous or next values
# Backward fill
df['Age'] = df['Age'].fillna(method='bfill')
# Forward fill
df['Age'] = df['Age'].fillna(method='ffill')

In [ ]:
df['Gender']=df['Gender'].fillna("Male")   # inserisco determinati valori (stringhe o numeri) al posto dei valori nulli
df['LoanAmount']=df['LoanAmount'].fillna(df['LoanAmount'].mean())   # inserisco la media della colonna al posto dei valori nulli

In [ ]:
df['Ultimo'] = df.Ultimo.str.replace(',', '.')  # ho una stringa e voglio modificarla
df['Ultimo']=pd.to_numeric(df['Ultimo'])        # passare da stringa a valore numerico

In [ ]:
# label encoder
df.replace({'Education':{'Graduate':1,'Not Graduate':0}, 'Property_Area':{'Semiurban':0,'Urban':1,"Rural":2},'Loan_Status':{'Y':1,'N':0}}, inplace=True)

###VARIABILI CATEGORICHE

In [ ]:
categorical_features = ['MSZoning','Neighborhood']
train_dummies = pd.get_dummies(train_df[categorical_features], drop_first=True)
X_train = pd.concat([X_train,train_dummies], axis=1, ignore_index=False)

test_dummies = pd.get_dummies(test_df[categorical_features], drop_first=True)
X_test = pd.concat([X_test,test_dummies], axis=1, ignore_index=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['target'] =le.fit_transform(df['target'])

#EDA

In [ ]:
### esplorazione target con displot
sns.displot(df["price"], height =7,aspect=2) 
plt.show()

In [ ]:
sns.countplot(x='smoker',data=tips);   # se ho variabili categoriche , posso vedere graificamente come sono distribuiti i valori
sns.countplot(x='sex',data=tips);

In [ ]:
### feature categoriche con la target

for feature in categorical_features:
  plt.figure(figsize=(16,9))
  sns.boxplot(x=feature, y=target_name, data=train_df)
  plt.title(feature)
  plt.xticks(rotation=90, fontsize=13);

In [ ]:
sns.relplot(x="total_bill", y="tip", hue="day",
                col="time", row="sex", data=tips);  # per estrapolare lo scatter 

In [ ]:
sns.pairplot(df, hue='class', height=3, aspect=1)  #  pairplot di tutte le features in relazione al target se è classifier uso anche hue altimenti no

In [ ]:
# matrice di correlazione
df.corr()

In [ ]:
import seaborn as sns
plt.figure(figsize=(30,26))
sns.heatmap(df.corr(),annot=True);   # heatmap con tutte le correlazioni

In [ ]:
# grafico di correlazione delle Features bar plot
price_corr = df.corr() ["price"].sort_values(ascending=False)

plt.figure(figsize=(10,7))
sns.barplot(x=price_corr.index, y=price_corr)
plt.xticks(rotation=90, fontsize=12)
plt.show()

In [ ]:
# features importances con random forest regressor
### fitting
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model = model.fit(train_df.drop([target_name]+list(categorical_features), axis=1), train_df[target_name])

In [ ]:
importances = pd.Series(model.feature_importances_, index=model.feature_names_in_).sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10,7))
sns.barplot(x=importances.index, y=importances)
plt.xticks(rotation=90, fontsize=12)
plt.show()

In [ ]:
# Definisco le Features e il Target 
X_ = df.drop(['Risk','LOCATION_ID'], axis=1)
y = df['Risk']

#STANDARDIZZAZIONE

In [ ]:
#standardizzazione
sc = StandardScaler()
standardized_data = sc.fit_transform(X_)
print(standardized_data)

In [ ]:
X = standardized_data

#SPLIT

In [ ]:
# Split the dataset into 70% Training and 30% Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=667
                                                    )

#RANDOM FOREST CLASSIFIER + FEATURE IMPORTANCE

In [ ]:
classifier = RandomForestClassifier(random_state=667, n_estimators=100,max_depth=10)

In [ ]:
#training the classifier
classifier.fit(X_train, y_train)

In [ ]:
#Feature importance for Random Forest
features = X_.columns
importances = classifier.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(18,10))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')

#MODELLI REGRESSION

In [ ]:
X_train = train_df[features].copy()
X_test = test_df[features].copy()

y_train = train_df[target_name].copy()
y_test = test_df[target_name].copy()

##Linear regression

In [ ]:
### LINEAR REGRESSION

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model = model.fit(X_train, y_train)

train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

### evaluation
from sklearn.metrics import r2_score

r2_train = r2_score(y_train, train_preds)
r2_test = r2_score(y_test, test_preds)

print(r2_train)
print(r2_test)

##K-NN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

r2_train_list = list()
r2_test_list = list()
for k in range(1,20):
    model = KNeighborsRegressor(n_neighbors=k)
    model = model.fit(X_train, y_train)

    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    ### evaluation
    from sklearn.metrics import r2_score

    r2_train = r2_score(y_train, train_preds)
    r2_test = r2_score(y_test, test_preds)

    r2_train_list += [r2_score(y_train, train_preds)]
    r2_test_list += [r2_score(y_test, test_preds)]

plt.plot(range(1,20), r2_train_list, label='TRAIN')
plt.plot(range(1,20), r2_test_list, label='TEST')
plt.xlabel('k-neighbors')
plt.ylabel('r2')
plt.legend()
plt.show()

In [ ]:
k=4
model = KNeighborsRegressor(n_neighbors=k)
model = model.fit(X_train, y_train)

train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

### evaluation
from sklearn.metrics import r2_score

print(r2_score(y_train, train_preds))
print(r2_score(y_test, test_preds))

##RANDOM FOREST

In [ ]:
### TUNING max_depth

from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

r2_train_list = list()
r2_test_list = list()
max_depth_range = range(1,20)

for d in tqdm(max_depth_range):
    model = RandomForestRegressor(max_depth=d)
    model = model.fit(X_train, y_train)

    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    ### evaluation
    from sklearn.metrics import r2_score

    r2_train = r2_score(y_train, train_preds)
    r2_test = r2_score(y_test, test_preds)

    r2_train_list += [r2_score(y_train, train_preds)]
    r2_test_list += [r2_score(y_test, test_preds)]

plt.plot(max_depth_range, r2_train_list, label='TRAIN')
plt.plot(max_depth_range, r2_test_list, label='TEST')
plt.xlabel('max_depth')
plt.ylabel('r2')
plt.legend()
plt.show()

In [ ]:
### TUNING n_estimators

r2_train_list = list()
r2_test_list = list()
n_estimators_range = range(1,20)

for n in n_estimators_range:
    model = RandomForestRegressor(n_estimators=n)
    model = model.fit(X_train, y_train)

    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    ### evaluation
    from sklearn.metrics import r2_score

    r2_train = r2_score(y_train, train_preds)
    r2_test = r2_score(y_test, test_preds)

    r2_train_list += [r2_score(y_train, train_preds)]
    r2_test_list += [r2_score(y_test, test_preds)]


plt.plot(n_estimators_range, r2_train_list, label='TRAIN')
plt.plot(n_estimators_range, r2_test_list, label='TEST')
plt.xlabel('n_estimators')
plt.ylabel('r2')
plt.legend()
plt.show()

In [ ]:
### final model

model = RandomForestRegressor(n_estimators=10, max_depth=7)
model = model.fit(X_train, y_train)

train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

### evaluation
from sklearn.metrics import r2_score

print(r2_score(y_train, train_preds))
print(r2_score(y_test, test_preds))

#FITTING CON I VARI MODELLI CLASSIFIER

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(max_depth=4)
dt_clf.fit(X_train, y_train)
dt_clf.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=200)
rf_clf.fit(X_train, y_train)
rf_clf.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)
gb_clf.score(X_test, y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test, y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB()
nb_clf.fit(X_train,y_train)
nb_clf.score(X_test, y_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=2)
knn_clf.fit(X_train,y_train)
knn_clf.score(X_test, y_test)

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC(probability=True)
svm_clf.fit(X_train,y_train)
svm_clf.score(X_test, y_test)

In [ ]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train,y_train)
xgb_clf.score(X_test, y_test)

##CONFUSION MATRIX DI TUTTI I MODELLI

In [ ]:
classifiers = [DecisionTreeClassifier(max_depth=4), 
               RandomForestClassifier(n_estimators=200),
               GradientBoostingClassifier(n_estimators=100),
               LogisticRegression(max_iter=1000),
               GaussianNB(),
               KNeighborsClassifier(n_neighbors=2),
               SVC(probability=True),
               XGBClassifier()
               ]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=667)

for cls in classifiers:
    cls.fit(X_train, y_train)
    cls.predict(X_test)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15,10))

for cls, ax in zip(classifiers, axes.flatten()):
    plot_confusion_matrix(cls, 
                          X_test, 
                          y_test, 
                          ax=ax, 
                          cmap='Blues',
                         display_labels=['morti','sopravv'])
    ax.title.set_text(type(cls).__name__)
plt.tight_layout()  
#plt.show()

#CLASSIFICATION REPORT

In [ ]:
from sklearn.metrics import classification_report
for cls, ax in zip(classifiers, axes.flatten()):
    print(cls)
    print(classification_report(y_test,cls.predict(X_test)))

#FEATURE SELECTION

In [ ]:
#FUTURE SELECTION CON BICLASSIFIER
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

featu=[]
for i in range(2,len(X_.columns)):
    rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=i)
    classifier = RandomForestClassifier(random_state=667, n_estimators=100,max_depth=8)
    pipe = Pipeline([
                    ('scaler', StandardScaler()),
                    ('featureselection',rfe),
                    ('classifier',classifier)
                    ])
    pipe.fit(X_train,y_train)
    test_y_pred = pipe.predict(X_test)
    test_acc = accuracy_score(y_test, test_y_pred)
    y_pred_proba = pipe.predict_proba(X_test)[::,1]
    roc_auc = roc_auc_score(y_test,y_pred_proba)
    featu.append(roc_auc)
    # summarize progress
    print('feature selected %d, Acc.Test: %.3f, AUC: %.3f' % (i, test_acc, roc_auc ))

print('\n Best number of feature selection is: %d' % (np.argmax(featu) +2))

In [ ]:
#FEATURE SELECTION CON MULTICLASSIFIER
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

featu=[]
for i in range(2,len(X_.columns)):
    rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=i)
    classifier = LogisticRegression(max_iter=10000)
    pipe = Pipeline([
                    ('scaler', StandardScaler()),
                    ('featureselection',rfe),
                    ('classifier',classifier)
                    ])
    pipe.fit(X_train,y_train)
    test_y_pred = pipe.predict(X_test)
    test_acc = accuracy_score(y_test, test_y_pred)
    f1 = f1_score(y_test, test_y_pred,average='macro')
    featu.append(f1)
    # summarize progress
    print('feature selected %d, Acc.Test: %.3f, F1: %.3f' % (i, test_acc, f1 ))

print('\n Best number of feature selection is: %d' % (np.argmax(featu) +2))

### Prima della Pipe line rifare lo split con X_

#PIPELINE

In [ ]:
rfe = RFE(estimator=LogisticRegression(max_iter=1000), n_features_to_select=5)
classifier = LogisticRegression(max_iter=1000)

pipe = Pipeline([
                ('scaler', StandardScaler()),
                ('featureselection',rfe),
                ('classifier',classifier)
                ])
pipe.fit(X_train,y_train)

#INPUT DATA

In [ ]:
input_data = [3,22,1,0,7.25,0,0,1,0,1]

prediction = pipe.predict([input_data])
print(prediction[0])

### Binary Classification ###
if (prediction[0]==0):
  print('Died')
else:
  print('Survived')

#JOBLIB

In [ ]:
#save pipeline model
import joblib
joblib.dump(pipe,'pipe_sonar.pkl')

In [ ]:
#Load model
uploaded_model = joblib.load('pipe_sonar.pkl')
uploaded_model.predict([input_data])[0] #0 The object is a Mine